### Related equations
(arxiv: https://arxiv.org/abs/cond-mat/0207529)

The distribution of real numbers $k$ (between $-Q$ and $Q$) and $\Lambda$ (between $-B$ and $B$) are derived in this script, where $0 < Q \leq \pi$ and $0 < B \leq \infty$. The distributions are normalized by
\begin{equation}
\int_{-Q}^{Q} \rho(k)dk = N/N_a \hspace{0.3cm} (\text{filling}) , \hspace{0.5cm} \int_{-B}^{B} \sigma(\Lambda)d\Lambda = M/N_a \hspace{0.3cm} (\text{density of down spin})
\end{equation}

The equations to solve $\rho(k)$ and $\sigma(\Lambda)$ are
\begin{equation}
\rho(k) = \frac{1}{2\pi}+\cos k\int_{-B}^{B} K(\sin k - \Lambda)\sigma(\Lambda)d\Lambda
\end{equation}
\begin{equation}
\sigma(\Lambda) = \int_{-Q}^{Q} K(\sin k -\Lambda)\rho(k)dk - \int_{-B}^{B} K^2(\Lambda-\Lambda')\sigma(\Lambda')d\Lambda'
\end{equation}
where
\begin{equation}
K(\Lambda - \Lambda') = \frac{1}{2\pi}\left[\frac{8U}{U^2 + 16(\Lambda-\Lambda')^2}\right]
\end{equation}
\begin{equation}
K^2(\Lambda - \Lambda') = \frac{1}{2\pi}\left[\frac{4U}{U^2 + 4(\Lambda-\Lambda')^2}\right]
\end{equation}
Half-filling and $S_z = 0$
$$Q = \pi, \hspace{0.5cm} B = \infty$$ 

Energy is
$$E(M, N) = -2N_a \int_{-Q}^{Q} \rho(k)\cos kdk$$

### Implementation
Rewrite integral equations of $\rho(k)$ and $\sigma(\Lambda)$ with descrete formula (matrix representation).
\begin{equation}
P_k = C^0_k + C_k * K_{kl}S_l
\end{equation}
\begin{equation}
S_l = K_{kl}P_k - K^2_{ll'}S_l
\end{equation}
where $\rho(k) \rightarrow P_k$, $\sigma(\Lambda) \rightarrow S_l$, $\frac{1}{2\pi} \rightarrow C^0_k$, $\cos k\rightarrow C_k$, $K(\sin k-\Lambda)\rightarrow K_{kl}$, and $K^2(\Lambda-\Lambda')\rightarrow K^2_{ll'}$.

Let  If we concatenate $P_k$ and $S_l$, we get
\begin{equation}
\begin{pmatrix}I & -CK \\ -K & I+K^2\end{pmatrix}
\begin{pmatrix}P \\ S\end{pmatrix} 
= \begin{pmatrix}C^0 \\ 0\end{pmatrix}
\end{equation}

In [15]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [42]:
class BatheAnsatzGS(object):
    def __init__(self,U, filling=0.5, nspind=0.25, ngrid=20, nstep = 50, tol=1e-5, maxiter=50):
        self.U = U
        self.filling = filling
        self.nspind = nspind
        # initialization 
        self.Q = np.pi 
        self.B = 20. 
        self.Qmax = np.pi
        self.Bmax = 20
        self.ngrid = ngrid
        self.kgrid = np.zeros(ngrid)
        self.lgrid = np.zeros(ngrid)
        self.rhok = np.zeros(ngrid)
        self.sigl = np.zeros(ngrid)
        self.tol = tol
        self.nstep = nstep
        self.maxiter = maxiter
        
    def K_func(self, k, l):
        U = self.U
        return 8.*U/(2.*np.pi*(U**2 + 16*(np.sin(k)-l)**2))
    def K2_func(self, x, xp):
        U = self.U
        return 4.*U/(2.*np.pi*(U**2 + 4*(x-xp)**2))
    def energy_per_site(self):
        return -2.*np.dot(self.rhok, np.cos(self.kgrid))
    def get_filling(self):
        return np.sum(self.rhok)
    def get_spindown(self):
        return np.sum(self.sigl)
    
    def solve_density(self):
        ngrid = self.ngrid
        Cmat = np.zeros((ngrid*2,)*2)
        Kmat = np.zeros((ngrid,ngrid))
        K2mat = np.zeros((ngrid,ngrid))
        for k in range(ngrid):
            for l in range(ngrid):
                Kmat[k,l] = self.K_func(self.kgrid[k], self.lgrid[l])
                K2mat[k,l] = self.K2_func(self.lgrid[k], self.lgrid[l])
        Cmat[:ngrid, :ngrid] = np.eye(ngrid)
        Cmat[:ngrid, ngrid:] = -np.diag(np.cos(self.kgrid)).dot(Kmat)
        Cmat[ngrid:, :ngrid] = -Kmat.T
        Cmat[ngrid:, ngrid:] = np.eye(ngrid)+K2mat
        
        const_vec = np.zeros(ngrid*2)
        const_vec[:ngrid] = np.ones(ngrid)/(2.*np.pi)
        results = np.linalg.solve(Cmat, const_vec)
        self.rhok = results[:ngrid]
        self.sigl = results[ngrid:]
        
    def kernel(self):
        Qmin = 0.01
        Bmin = 0
        nstep = self.nstep
        self.Q = Qmin
        self.B = Bmin
        dQ = (self.Qmax - Qmin)/nstep
        dB = (self.Bmax - Bmin)/nstep
        for i in range(self.maxiter):
            self.kgrid = np.linspace(-self.Q, self.Q, self.ngrid)
            self.lgrid = np.linspace(-self.B, self.B, self.ngrid)
            self.solve_density()
            tmp_fill = self.get_filling()
            tmp_spind = self.get_spindown()
            diff = (abs(tmp_fill-self.filling)+abs(tmp_spind-self.nspind))/2
            print diff
            if(diff < self.tol):
                print "The calculation converged with %d loops!\n"%(i+1)
                break
            if (i==self.maxiter-1):
                print "The calculation did not converge after %d loops!\n"%(self.maxiter)
            self.Q += dQ
            self.B += dB
        energy = self.energy_per_site()
        return energy

In [43]:
U = 4
ba = BatheAnsatzGS(U, filling=0.5, nspind=0.25, ngrid=20, nstep = 50, tol=1e-5, maxiter=50)

In [44]:
ba.kernel()

0.836986652583
0.838899370856
0.843618380731
0.850915353439
0.860460209175
0.871862168133
0.884712996697
0.898625284727
0.913261197074
0.928350128346
0.943696111605
0.959177260549
0.974739995487
0.990390602587
1.00618613701
1.02222606518
1.03864550495
1.05561052935
1.07331575924
1.09198436205
1.11187057519
1.13326496297
1.15650278934
1.18197615371
1.21015092367
1.24159006376
1.27698580806
1.31720443335
1.36334947316
1.41685263484
1.47960748986
1.55417123022
1.64407851649
1.7543473927
1.89233002779
2.069218045
2.30287772662
2.62362552285
3.08725190645
3.80873003573
5.06877518801
7.78047299884
17.566179522
60.1176842875
10.9777952584
6.05068291004
4.20213054885
3.24563116119
2.66840531877
2.28698352596
The calculation did not converge after 50 loops!



-14.485963600019154